In [15]:
import os
import pandas as pd
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed


In [16]:
response_74 = {
    'resultado':[0, 1, 0, 1, 1],
    'entidade': [[0], ['A', 'B'], [0], ['C'], ['D', 'E', 'F']],
    'codigo_palavra': [[0], [15, 25], [0], [500], [125, 86, 15]],
    'mensagem': ['Execucao correta do 74'],
    'status': [200]
}

len(response_74['status'])

1

In [17]:
response_75 = {
    'resultado':[0, 0, 1, 1, 1],
    'entidade': [[0], [0], ['G'], ['H'], ['I', 'J', 'K']],
    'codigo_palavra': [[0], [0], [614], [600], [600, 614, 754]],
    'mensagem': ['Execucao correta do 75'],
    'status': [200]
}

len(response_75)

5

In [18]:
dados = {
    'PRTL': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010],
    'CD_CLI': [25, 30, 45, 60, 74, 54, 10, 21, 11, 36],
    'TXT_MSG': ['Z', 'P', 'Q', 'U', 'M', 'V', 'T', 'X', 'Y', 'W']
}

corpus = pd.DataFrame(dados)
corpus
print(len(corpus))

10


In [19]:
coluna = 'ENT'
corpus[coluna] = json.dumps({'Entidades': []})
corpus

,PRTL,CD_CLI,TXT_MSG,ENT
0,1001,25,Z,"{""Entidades"": []}"
1,1002,30,P,"{""Entidades"": []}"
2,1003,45,Q,"{""Entidades"": []}"
3,1004,60,U,"{""Entidades"": []}"
4,1005,74,M,"{""Entidades"": []}"
5,1006,54,V,"{""Entidades"": []}"
6,1007,10,T,"{""Entidades"": []}"
7,1008,21,X,"{""Entidades"": []}"
8,1009,11,Y,"{""Entidades"": []}"
9,1010,36,W,"{""Entidades"": []}"


In [20]:
def dividir_dataframe(df, num_linhas):
    num_chunks = len(df) // num_linhas + int(len(df) % num_linhas != 0)
    print(num_chunks)

    return np.array_split(df, num_chunks)

resultados = dividir_dataframe(corpus, 5)

2


/home/pgmadmin/.asdf/installs/python/3.11.5/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [21]:
def inferencia_ner(chunk_, indicador_, coluna_):
    chunk_.reset_index(drop=True, inplace=True)

    # simula o resultado do requests.post
    data_json = response_75
    num_msg = len(chunk_)

    
    for i in range(num_msg):
        ent_json = {
            'indicador': indicador_,
            'palavras': data_json['entidade'][i],
            'cod_palavra':data_json['codigo_palavra'][i]
        }
        try:
            temp = chunk_.loc[i, coluna_]
            temp_json = json.loads(temp)
            temp_json['Entidades'].append(ent_json)
            chunk_.loc[i, coluna_] = json.dumps(temp_json)
        except Exception as err:
            print(f'Erro ao tentar inserir ent_json na linha {i}: [err]')
        
    return chunk_

############
workers=1


results = []

with ThreadPoolExecutor(max_workers=workers) as executor:
    # COD 74
    try:
        #print(resultados[0])    
        futures = {executor.submit(inferencia_ner, chunk, 74, coluna): chunk for chunk in resultados}
        
        for future in as_completed(futures):
            try:
                data = future.result()
                results.append(data)
            except Exception as exc:
                print(f'Generated an exception: {exc}')
        #print(f'\nResultado total: \n{results}\n')
    except Exception as err:
        print(f'Erro fo executor: {err}')

resultados = []
resultados = results
results = []

    # COD 75
with ThreadPoolExecutor(max_workers=workers) as executor:
        # COD 75
    try:
        #print(resultados[0])    
        futures = {executor.submit(inferencia_ner, chunk, 75, coluna): chunk for chunk in resultados}
        
        for future in as_completed(futures):
            chunk = futures[future]
            try:
                data = future.result()
                results.append(data)
            except Exception as exc:
                print(f'Generated an exception: chunk:{chunk} |\n  {exc}')
        #print(f'\nResultado total: \n{results}\n')
    except Exception as err:
        print(f'Erro fo executor: {err}')

resultados = []
resultados = results
results = []      
        
        
final = pd.concat(resultados, ignore_index=True)
print(f'\nresultado montado: \n{final}')
final.to_csv('result.csv', index=False)


resultado montado: 
   PRTL  CD_CLI TXT_MSG                                                ENT
0  1001      25       Z  {"Entidades": [{"indicador": 74, "palavras": [...
1  1002      30       P  {"Entidades": [{"indicador": 74, "palavras": [...
2  1003      45       Q  {"Entidades": [{"indicador": 74, "palavras": [...
3  1004      60       U  {"Entidades": [{"indicador": 74, "palavras": [...
4  1005      74       M  {"Entidades": [{"indicador": 74, "palavras": [...
5  1006      54       V  {"Entidades": [{"indicador": 74, "palavras": [...
6  1007      10       T  {"Entidades": [{"indicador": 74, "palavras": [...
7  1008      21       X  {"Entidades": [{"indicador": 74, "palavras": [...
8  1009      11       Y  {"Entidades": [{"indicador": 74, "palavras": [...
9  1010      36       W  {"Entidades": [{"indicador": 74, "palavras": [...
